In [ ]:
import numpy as np # Linear Algebra
import matplotlib.pyplot as plt # Plotting

In [ ]:
# Full Extended Kalman filter N-D
def kalman_step(mu, sig, u, z, f, df, h, dh, R = np.eye(2) * 0.1, Q = np.eye(2) * 0.1):
    mu_bar = f(mu, u)
    F = df(mu, u)
    sig_bar = F @ sig @ F.T + R
    H = dh(mu_bar)
    K = sig_bar @ H.T @ np.linalg.inv(H @ sig_bar @ H.T + Q)
    mu_new = mu_bar + K @ (z - h(mu_bar))
    sig_new = (np.eye(*sig.shape) - K @ H) @ sig_bar
    return mu_new, sig_new

In [ ]:
# Dynamics
def pendulum_dynamics(x, u):
    gravity = 9.81
    length = 1.
    mass = 1.
    x_dot = np.concatenate([x[1:], -(gravity/length)*np.sin(x[:1]) + 1./(mass*length*length)*u], axis=0)
    return x_dot

def pendulum_euler(x, u):
    dt = 0.01
    return x + pendulum_dynamics(x, u) * dt

In [ ]:
# Jacobians
def pendulum_deriv(x, u):
    gravity = 9.81
    length = 1.
    mass = 1.
    return np.array([[0., 1.], [-(gravity/length)*np.cos(x[0, 0]), 0.]])

def pendulum_euler_deriv(x, u):
    dt = 0.01
    return np.eye(2) + pendulum_deriv(x, u) * dt

In [ ]:
# Observations: we can see the tip of the pendulum
def tip(x):
    length = 1.
    px = length * np.sin(np.pi - x[0, 0])
    py = length * np.cos(np.pi - x[0, 0])

    return np.array([[px], [py]])

def tip_deriv(x):
    length = 1.
    return np.array([[-length * np.cos(np.pi - x[0, 0]), 0.], [length * np.sin(np.pi - x[0, 0]), 0.]])

In [ ]:
R = np.eye(2) * 0.001 # noise for movement! (1mm variance!)
Q = np.eye(2) * 0.01 # noise for observation! (1cm variance!)

In [ ]:
# Let's do a simulation!
x = np.zeros((2, 1)) # We start at (0,0)
# x[0, 0] = -0.5
u = 4.

K = 400 # K steps

states = [np.copy(x)]
estimation = [(np.copy(x), np.eye(2) * 0.001)] # We start with a small uncertainty!

for k in range(K):
    # u = np.array([[0.05 * np.sin(2.*k*dt)], [-0.1 * np.sin(2.*k*dt)]])
    # Let's move the simulator forward
    x = pendulum_euler(x, u)
    states.append(np.copy(x))
    # Let's get the measurement!
    z = tip(x) + np.random.multivariate_normal(np.zeros((2,)), Q) # with noise!
    # Let's do a Kalman step!
    mu, sig = estimation[-1]
    mu_new, sig_new = kalman_step(mu, sig, u, z, pendulum_euler, pendulum_euler_deriv, tip, tip_deriv, R, Q)
    estimation.append((np.copy(mu_new), np.copy(sig_new)))

In [ ]:
# Let's plot!
times = np.arange(len(states)) * 0.01

# Positions
plt.plot(times, [st[0] for st in states], label='θ atual')
plt.plot(times, [st[0][0, 0] for st in estimation], label='θ estimation')

plt.plot(times, [st[1] for st in states], label='$\dot{θ}$ actual')
plt.plot(times, [st[0][1, 0] for st in estimation], label='$\dot{θ}$ estimation')


plt.legend()

plt.show()